<a href="https://colab.research.google.com/github/sulmannizami/Food-Recommendation-System-for-Sugar-Patients-diabetics-/blob/main/Food_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas numpy scikit-learn tensorflow torch flask


In [2]:
# food_recommendation_system.py

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler

# Step 1: Data Preparation
# Example data structure for foods and patient details
data = {
    'food_item': ['Apple', 'Banana', 'Rice', 'Chicken', 'Spinach'],
    'calories': [52, 89, 130, 239, 23],
    'sugar_content': [10, 12, 0, 0, 0.4],
    'glycemic_index': [38, 51, 73, 0, 15],  # Lower GI is better for sugar patients
    'is_vegetarian': [True, True, False, False, True]
}

# Create DataFrame
df = pd.DataFrame(data)

# Patient dataset containing age, sex, and medical history (e.g., whether the patient has diabetes, hypertension, etc.)
patient_data = {
    'age': [30, 50, 40, 60, 35],
    'sex': ['M', 'F', 'M', 'F', 'M'],  # M for Male, F for Female
    'has_diabetes': [True, True, False, True, False],
    'has_hypertension': [False, True, True, False, False]
}

# Create DataFrame for patient data
patient_df = pd.DataFrame(patient_data)

# Step 2: Preprocessing and Feature Engineering
# Standardizing the numerical features (calories, sugar_content, glycemic_index)
scaler = StandardScaler()
df[['calories', 'sugar_content', 'glycemic_index']] = scaler.fit_transform(df[['calories', 'sugar_content', 'glycemic_index']])

# Converting categorical data (sex) into numeric form (e.g., M -> 1, F -> 0)
patient_df['sex'] = patient_df['sex'].apply(lambda x: 1 if x == 'M' else 0)

# Step 3: PyTorch Model for Food Recommendation (Extended to Include Patient Details)

class FoodRecommendationModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FoodRecommendationModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# PyTorch Model Training
def pytorch_model_training(df, patient_df):
    # Combining food features with patient features (age, sex, medical history)
    X_food = torch.tensor(df[['calories', 'sugar_content', 'glycemic_index']].values, dtype=torch.float32)
    X_patient = torch.tensor(patient_df[['age', 'sex', 'has_diabetes', 'has_hypertension']].values, dtype=torch.float32)

    # Concatenate food features and patient features
    X = torch.cat((X_food, X_patient), dim=1)

    # Example labels for classification (index of food items)
    y = torch.tensor([0, 1, 2, 3, 4], dtype=torch.long)

    # Initialize model, loss, and optimizer
    model = FoodRecommendationModel(input_size=7, hidden_size=10, output_size=len(df))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    num_epochs = 100
    for epoch in range(num_epochs):
        # Forward pass
        outputs = model(X)
        loss = criterion(outputs, y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    return model

# Predict using PyTorch model
def pytorch_predict(model, new_input_food, new_input_patient):
    new_input_food = torch.tensor(new_input_food, dtype=torch.float32)
    new_input_patient = torch.tensor(new_input_patient, dtype=torch.float32)

    # Concatenate food features and patient features
    new_input = torch.cat((new_input_food, new_input_patient), dim=1)

    predicted_food_index = torch.argmax(model(new_input)).item()
    return df['food_item'].iloc[predicted_food_index]

# Step 4: TensorFlow Model for Food Recommendation (Extended to Include Patient Details)

def tensorflow_model_training(df, patient_df):
    # Combining food features with patient features (age, sex, medical history)
    X_food = df[['calories', 'sugar_content', 'glycemic_index']].values
    X_patient = patient_df[['age', 'sex', 'has_diabetes', 'has_hypertension']].values
    X = pd.concat([pd.DataFrame(X_food), pd.DataFrame(X_patient)], axis=1).values

    # Example labels for classification
    y = [0, 1, 2, 3, 4]  # Index of food items

    # Define the model
    model_tf = Sequential([
        Dense(10, input_dim=7, activation='relu'),
        Dense(len(df), activation='softmax')  # Output layer for classification
    ])

    # Compile the model
    model_tf.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model_tf.fit(X, y, epochs=100)

    return model_tf

# Predict using TensorFlow model
def tensorflow_predict(model_tf, new_input_food, new_input_patient):
    new_input = new_input_food + new_input_patient
    predicted_food_index_tf = model_tf.predict([new_input]).argmax()
    return df['food_item'].iloc[predicted_food_index_tf]

# Step 5: Example Usage

# Input: New patient details for prediction (age, sex, medical history) and new food details
new_input_food = [70, 5, 45]  # Example: New food item (calories, sugar content, glycemic index)
new_input_patient = [45, 1, 1, 0]  # Example: Patient details (age, sex, diabetes, hypertension)

# PyTorch Prediction
model_pytorch = pytorch_model_training(df, patient_df)
print("\n[PyTorch] Recommended Food:", pytorch_predict(model_pytorch, [new_input_food], [new_input_patient]))

# TensorFlow Prediction
model_tf = tensorflow_model_training(df, patient_df)
print("\n[TensorFlow] Recommended Food:", tensorflow_predict(model_tf, new_input_food, new_input_patient))


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [3]:
# food_recommendation_system.py

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler

# Step 1: Data Preparation
# Example data structure for foods and patient details
data = {
    'food_item': ['Apple', 'Banana', 'Rice', 'Chicken', 'Spinach'],
    'calories': [52, 89, 130, 239, 23],
    'sugar_content': [10, 12, 0, 0, 0.4],
    'glycemic_index': [38, 51, 73, 0, 15],  # Lower GI is better for sugar patients
    'is_vegetarian': [True, True, False, False, True]
}

# Create DataFrame
df = pd.DataFrame(data)

# Patient dataset containing age, sex, and medical history (e.g., whether the patient has diabetes, hypertension, etc.)
patient_data = {
    'age': [30, 50, 40, 60, 35],
    'sex': ['M', 'F', 'M', 'F', 'M'],  # M for Male, F for Female
    'has_diabetes': [True, True, False, True, False],
    'has_hypertension': [False, True, True, False, False]
}

# Create DataFrame for patient data
patient_df = pd.DataFrame(patient_data)

# Step 2: Preprocessing and Feature Engineering
# Standardizing the numerical features (calories, sugar_content, glycemic_index)
scaler = StandardScaler()
df[['calories', 'sugar_content', 'glycemic_index']] = scaler.fit_transform(df[['calories', 'sugar_content', 'glycemic_index']])

# Converting categorical data (sex) into numeric form (e.g., M -> 1, F -> 0)
patient_df['sex'] = patient_df['sex'].apply(lambda x: 1 if x == 'M' else 0)

# Step 3: PyTorch Model for Food Recommendation (Extended to Include Patient Details)

class FoodRecommendationModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FoodRecommendationModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# PyTorch Model Training
def pytorch_model_training(df, patient_df):
    # Combining food features with patient features (age, sex, medical history)
    X_food = torch.tensor(df[['calories', 'sugar_content', 'glycemic_index']].values, dtype=torch.float32)
    X_patient = torch.tensor(patient_df[['age', 'sex', 'has_diabetes', 'has_hypertension']].values, dtype=torch.float32)

    # Concatenate food features and patient features
    X = torch.cat((X_food, X_patient), dim=1)

    # Example labels for classification (index of food items)
    y = torch.tensor([0, 1, 2, 3, 4], dtype=torch.long)

    # Initialize model, loss, and optimizer
    model = FoodRecommendationModel(input_size=7, hidden_size=10, output_size=len(df))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    num_epochs = 100
    for epoch in range(num_epochs):
        # Forward pass
        outputs = model(X)
        loss = criterion(outputs, y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    return model

# Predict using PyTorch model
def pytorch_predict(model, new_input_food, new_input_patient):
    new_input_food = torch.tensor(new_input_food, dtype=torch.float32)
    new_input_patient = torch.tensor(new_input_patient, dtype=torch.float32)

    # Concatenate food features and patient features
    new_input = torch.cat((new_input_food, new_input_patient), dim=1)

    predicted_food_index = torch.argmax(model(new_input)).item()
    return df['food_item'].iloc[predicted_food_index]

# Step 4: TensorFlow Model for Food Recommendation (Extended to Include Patient Details)

def tensorflow_model_training(df, patient_df):
    # Combining food features with patient features (age, sex, medical history)
    X_food = df[['calories', 'sugar_content', 'glycemic_index']].values
    X_patient = patient_df[['age', 'sex', 'has_diabetes', 'has_hypertension']].values
    X = pd.concat([pd.DataFrame(X_food), pd.DataFrame(X_patient)], axis=1).values

    # Example labels for classification
    y = [0, 1, 2, 3, 4]  # Index of food items

    # Define the model
    model_tf = Sequential([
        Dense(10, input_dim=7, activation='relu'),
        Dense(len(df), activation='softmax')  # Output layer for classification
    ])

    # Compile the model
    model_tf.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model_tf.fit(X, y, epochs=100)

    return model_tf

# Predict using TensorFlow model
def tensorflow_predict(model_tf, new_input_food, new_input_patient):
    new_input = new_input_food + new_input_patient
    predicted_food_index_tf = model_tf.predict([new_input]).argmax()
    return df['food_item'].iloc[predicted_food_index_tf]

# Step 5: Example Usage

# Input: New patient details for prediction (age, sex, medical history) and new food details
new_input_food = [70, 5, 45]  # Example: New food item (calories, sugar content, glycemic index)
new_input_patient = [45, 1, 1, 0]  # Example: Patient details (age, sex, diabetes, hypertension)

# PyTorch Prediction
model_pytorch = pytorch_model_training(df, patient_df)
print("\n[PyTorch] Recommended Food:", pytorch_predict(model_pytorch, [new_input_food], [new_input_patient]))

# TensorFlow Prediction
model_tf = tensorflow_model_training(df, patient_df)
print("\n[TensorFlow] Recommended Food:", tensorflow_predict(model_tf, new_input_food, new_input_patient))


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [4]:
# food_recommendation_system.py

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler

# Step 1: Data Preparation
# Example data structure for foods and patient details
data = {
    'food_item': ['Apple', 'Banana', 'Rice', 'Chicken', 'Spinach'],
    'calories': [52, 89, 130, 239, 23],
    'sugar_content': [10, 12, 0, 0, 0.4],
    'glycemic_index': [38, 51, 73, 0, 15],  # Lower GI is better for sugar patients
    'is_vegetarian': [True, True, False, False, True]
}

# Create DataFrame
df = pd.DataFrame(data)

# Patient dataset containing age, sex, and medical history (e.g., whether the patient has diabetes, hypertension, etc.)
patient_data = {
    'age': [30, 50, 40, 60, 35],
    'sex': ['M', 'F', 'M', 'F', 'M'],  # M for Male, F for Female
    'has_diabetes': [True, True, False, True, False],
    'has_hypertension': [False, True, True, False, False]
}

# Create DataFrame for patient data
patient_df = pd.DataFrame(patient_data)

# Step 2: Preprocessing and Feature Engineering
# Standardizing the numerical features (calories, sugar_content, glycemic_index)
scaler = StandardScaler()
df[['calories', 'sugar_content', 'glycemic_index']] = scaler.fit_transform(df[['calories', 'sugar_content', 'glycemic_index']])

# Converting categorical data (sex) into numeric form (e.g., M -> 1, F -> 0)
patient_df['sex'] = patient_df['sex'].apply(lambda x: 1 if x == 'M' else 0)

# Step 3: PyTorch Model for Food Recommendation (Extended to Include Patient Details)

class FoodRecommendationModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FoodRecommendationModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# PyTorch Model Training
def pytorch_model_training(df, patient_df):
    # Combining food features with patient features (age, sex, medical history)
    X_food = torch.tensor(df[['calories', 'sugar_content', 'glycemic_index']].values, dtype=torch.float32)
    X_patient = torch.tensor(patient_df[['age', 'sex', 'has_diabetes', 'has_hypertension']].values, dtype=torch.float32)

    # Concatenate food features and patient features
    X = torch.cat((X_food, X_patient), dim=1)

    # Example labels for classification (index of food items)
    y = torch.tensor([0, 1, 2, 3, 4], dtype=torch.long)

    # Initialize model, loss, and optimizer
    model = FoodRecommendationModel(input_size=7, hidden_size=10, output_size=len(df))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    num_epochs = 100
    for epoch in range(num_epochs):
        # Forward pass
        outputs = model(X)
        loss = criterion(outputs, y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    return model

# Predict using PyTorch model
def pytorch_predict(model, new_input_food, new_input_patient):
    new_input_food = torch.tensor(new_input_food, dtype=torch.float32)
    new_input_patient = torch.tensor(new_input_patient, dtype=torch.float32)

    # Concatenate food features and patient features
    new_input = torch.cat((new_input_food, new_input_patient), dim=1)

    predicted_food_index = torch.argmax(model(new_input)).item()
    return df['food_item'].iloc[predicted_food_index]

# Step 4: Example Usage

# Input: New patient details for prediction (age, sex, medical history) and new food details
new_input_food = [70, 5, 45]  # Example: New food item (calories, sugar content, glycemic index)
new_input_patient = [45, 1, 1, 0]  # Example: Patient details (age, sex, diabetes, hypertension)

# PyTorch Prediction
model_pytorch = pytorch_model_training(df, patient_df)
print("\n[PyTorch] Recommended Food:", pytorch_predict(model_pytorch, [new_input_food], [new_input_patient]))


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [5]:
#  Food Recommendation System for Sugar Patients

# Food list based on sugar content and medical conditions
foods = {
    "Apple": {"sugar_content": "Low", "vegetarian": True},
    "Banana": {"sugar_content": "High", "vegetarian": True},
    "Chicken": {"sugar_content": "Low", "vegetarian": False},
    "Spinach": {"sugar_content": "Very Low", "vegetarian": True},
    "Rice": {"sugar_content": "High", "vegetarian": False}
}

# Function to recommend food based on medical conditions and preferences
def recommend_food(age, sex, has_diabetes, has_hypertension, is_vegetarian):
    recommended_foods = []

    # Suggest foods based on diabetes and vegetarian preference
    for food, details in foods.items():
        if has_diabetes and details["sugar_content"] == "Low" or details["sugar_content"] == "Very Low":
            if is_vegetarian == details["vegetarian"]:
                recommended_foods.append(food)

    if not recommended_foods:
        recommended_foods.append("No suitable food found for your preferences.")

    return recommended_foods

# Taking input from the user (patient details)
age = int(input("Enter age: "))
sex = input("Enter sex (M/F): ")
has_diabetes = input("Do you have diabetes? (yes/no): ").lower() == 'yes'
has_hypertension = input("Do you have hypertension? (yes/no): ").lower() == 'yes'
is_vegetarian = input("Are you vegetarian? (yes/no): ").lower() == 'yes'

# Get food recommendation
recommended_foods = recommend_food(age, sex, has_diabetes, has_hypertension, is_vegetarian)

# Output the recommended foods
print("\nRecommended Foods for You:")
for food in recommended_foods:
    print(food)


Enter age: 32
Enter sex (M/F): M
Do you have diabetes? (yes/no): no
Do you have hypertension? (yes/no): no
Are you vegetarian? (yes/no): yes

Recommended Foods for You:
Spinach
